In [1]:
#HA#10
#Paula Abigail Tam <921850992>
#CSC620

#HA#9 start-----
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df = pd.read_csv('./HA9_csv/train.csv') #My path to the train.csv file is this
#('../input/train.csv') 

df.dropna(axis=0) #.dropna removes any missing values, so there is no empty values in the data
    #documentation: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html
df.set_index('id', inplace = True)

import re #regex
import nltk
from nltk.corpus import stopwords
#imports to get the part-of-speech tags and tokenizer
from nltk import pos_tag
from nltk import word_tokenize

stopWords = set(stopwords.words('english')) #makes a set of the stopwords 

#creating a function to encapsulate preprocessing, to make it easy to replicate on  submission data
def processing(df):
    #lowering and removing punctuation
    df['processed'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]','', x.lower()))
    #numerical feature engineering
    #total length of sentence
    df['length'] = df['processed'].apply(lambda x: len(x))
    #get number of words
    df['words'] = df['processed'].apply(lambda x: len(x.split(' '))) #total words
    df['words_not_stopword'] = df['processed'].apply(lambda x: len([t for t in x.split(' ') if t not in stopWords])) #words that arent stopwords
    #get the average word length
    df['avg_word_length'] = df['processed'].apply(lambda x: np.mean([len(t) for t in x.split(' ') if t not in stopWords]) if len([len(t) for t in x.split(' ') if t not in stopWords]) > 0 else 0)
    df['commas'] = df['text'].apply(lambda x: x.count(',')) #count the number of commas in the text
    df['pos'] = df['processed'].apply(lambda x: nltk.pos_tag(word_tokenize(x)))
    return(df)

df = processing(df)

#adding the 3 features: noun, adj, verb
def process_pos(df, pos): 
    full_list = []
    for row in df['pos']:
        #new_list = [] #initially had a list of tuples
        count = 0
        for key, val in row:
            if val == pos: #if the value matches with the given string (e.g. 'JJ', NN', 'VB')
                #new_list.append((key, val)) #put tuple with the word (key) and pos (val)
                count += 1 #increment count
        full_list.append(count) #put whole list of that row
    return full_list #returns list of lengths of the number of whatever pos was specified

pos_adj = process_pos(df, 'JJ') #JJ is adjective
pos_n = process_pos(df, 'NN') #NN is noun
pos_v = process_pos(df, 'VB') #VB is verb

#making the new columns for our new features
df['adjectives'] = pos_adj
df['nouns'] = pos_n
df['verbs'] = pos_v

df.head()

,text,author,processed,length,words,words_not_stopword,avg_word_length,commas,pos,adjectives,nouns,verbs
id,,,,,,,,,,,,
id26305,"This process, however, afforded me no means of...",EAP,this process however afforded me no means of a...,224,41,21,6.380952,4,"[(this, DT), (process, NN), (however, RB), (af...",2,10,1
id17569,It never once occurred to me that the fumbling...,HPL,it never once occurred to me that the fumbling...,70,14,6,6.166667,0,"[(it, PRP), (never, RB), (once, RB), (occurred...",1,2,1
id11008,"In his left hand was a gold snuff box, from wh...",EAP,in his left hand was a gold snuff box from whi...,195,36,19,5.947368,4,"[(in, IN), (his, PRP$), (left, JJ), (hand, NN)...",4,9,0
id27763,How lovely is spring As we looked from Windsor...,MWS,how lovely is spring as we looked from windsor...,202,34,21,6.476190,3,"[(how, WRB), (lovely, RB), (is, VBZ), (spring,...",5,6,0
id12958,"Finding nothing else, not even gold, the Super...",HPL,finding nothing else not even gold the superin...,170,27,16,7.187500,2,"[(finding, VBG), (nothing, NN), (else, RB), (n...",1,5,1


In [2]:
from sklearn.model_selection import train_test_split

features= [c for c in df.columns.values if c not in ['id','text','author', 'pos']] #everything else other than the stuff we dont care about (not data)
numeric_features= [c for c in df.columns.values if c not in ['id','text','author','processed', 'pos']] #the features with numeric values
target = 'author'
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.33, random_state=42) #test will be 1/3 and train will be 2/3
X_train.head()

,processed,length,words,words_not_stopword,avg_word_length,commas,adjectives,nouns,verbs
id,,,,,,,,,
id19417,this panorama is indeed glorious and i should ...,91,18,6,6.666667,1,1,4,1
id09522,there was a simple natural earnestness about h...,240,44,18,6.277778,4,6,6,0
id22732,who are you pray that i duc de lomelette princ...,387,74,38,5.552632,9,1,15,4
id10351,he had gone in the carriage to the nearest tow...,118,24,11,5.363636,0,0,6,0
id24580,there is no method in their proceedings beyond...,71,13,5,7.000000,1,0,3,0


In [3]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
    def get_feature_names(self):
        return X[self.key].columns.tolist()

    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
    def get_feature_names(self):
        return x[[self.key]].columns.tolist()

    

In [4]:
#all pipelines are here
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

text = Pipeline([
                ('selector', TextSelector(key='processed')),
                ('tfidf', TfidfVectorizer( stop_words='english'))
            ])

text.fit_transform(X_train)

length =  Pipeline([
                ('selector', NumberSelector(key='length')),
                ('standard', StandardScaler())
            ])

length.fit_transform(X_train)

words =  Pipeline([
                ('selector', NumberSelector(key='words')),
                ('standard', StandardScaler())
            ])
words_not_stopword =  Pipeline([
                ('selector', NumberSelector(key='words_not_stopword')),
                ('standard', StandardScaler())
            ])
avg_word_length =  Pipeline([
                ('selector', NumberSelector(key='avg_word_length')),
                ('standard', StandardScaler())
            ])
commas =  Pipeline([
                ('selector', NumberSelector(key='commas')),
                ('standard', StandardScaler()),
            ])
adjectives =  Pipeline([
                ('selector', NumberSelector(key='adjectives')),
                ('standard', StandardScaler()),
            ])
nouns =  Pipeline([
                ('selector', NumberSelector(key='nouns')),
                ('standard', StandardScaler()),
            ])
verbs =  Pipeline([
                ('selector', NumberSelector(key='verbs')),
                ('standard', StandardScaler()),
            ])

In [5]:
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

feats = FeatureUnion([('text', text), 
                      ('length', length),
                      ('words', words),
                      ('words_not_stopword', words_not_stopword),
                      ('avg_word_length', avg_word_length),
                      ('commas', commas),
                      ('adjectives', adjectives),
                      ('nouns', nouns),
                      ('verbs', verbs)])

feature_processing = Pipeline([('feats', feats)])
feature_processing.fit_transform(X_train)

pipeline = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42, max_iter=1000)),
])

pipeline.fit(X_train, y_train)

preds = pipeline.predict(X_test)
print(np.mean(preds == y_test))
print(classification_report(y_test, preds))

#HA#9 end-----

0.7862890745899103
              precision    recall  f1-score   support

         EAP       0.74      0.85      0.79      2587
         HPL       0.81      0.74      0.78      1852
         MWS       0.84      0.74      0.79      2023

    accuracy                           0.79      6462
   macro avg       0.80      0.78      0.79      6462
weighted avg       0.79      0.79      0.79      6462



In [6]:
#this portion is checking for the errors --> error analysis
test_array = pipeline.predict_proba(X_test)
test_array

array([[0.76101144, 0.11418528, 0.12480328],
       [0.26860046, 0.18050282, 0.55089672],
       [0.09488275, 0.62131342, 0.28380383],
       ...,
       [0.07531886, 0.07514066, 0.84954048],
       [0.66923716, 0.26546624, 0.0652966 ],
       [0.54973872, 0.08000664, 0.37025463]])

In [7]:
def err_analysis(arr):
    gt_list = []
    for row in arr:
        max_val = max(row)
        if max_val == row[0]:
            gt_list.append('EAP')
        elif max_val == row[1]:
            gt_list.append('HPL')
        elif max_val == row[2]:
            gt_list.append('MWS')
    return gt_list

err_check = err_analysis(test_array)

In [8]:
err_check

['EAP',
 'MWS',
 'HPL',
 'EAP',
 'HPL',
 'EAP',
 'MWS',
 'EAP',
 'EAP',
 'EAP',
 'HPL',
 'EAP',
 'MWS',
 'EAP',
 'HPL',
 'EAP',
 'EAP',
 'MWS',
 'EAP',
 'MWS',
 'EAP',
 'EAP',
 'MWS',
 'EAP',
 'MWS',
 'EAP',
 'EAP',
 'MWS',
 'EAP',
 'HPL',
 'EAP',
 'HPL',
 'MWS',
 'MWS',
 'MWS',
 'EAP',
 'MWS',
 'MWS',
 'MWS',
 'EAP',
 'EAP',
 'EAP',
 'MWS',
 'MWS',
 'EAP',
 'EAP',
 'EAP',
 'HPL',
 'HPL',
 'MWS',
 'MWS',
 'MWS',
 'MWS',
 'EAP',
 'EAP',
 'EAP',
 'HPL',
 'HPL',
 'HPL',
 'HPL',
 'EAP',
 'HPL',
 'EAP',
 'MWS',
 'EAP',
 'EAP',
 'MWS',
 'MWS',
 'HPL',
 'HPL',
 'EAP',
 'EAP',
 'EAP',
 'EAP',
 'EAP',
 'MWS',
 'MWS',
 'HPL',
 'EAP',
 'EAP',
 'HPL',
 'EAP',
 'MWS',
 'EAP',
 'MWS',
 'HPL',
 'EAP',
 'MWS',
 'HPL',
 'EAP',
 'MWS',
 'EAP',
 'EAP',
 'MWS',
 'MWS',
 'EAP',
 'EAP',
 'MWS',
 'MWS',
 'EAP',
 'HPL',
 'MWS',
 'EAP',
 'MWS',
 'HPL',
 'MWS',
 'MWS',
 'MWS',
 'EAP',
 'EAP',
 'EAP',
 'MWS',
 'EAP',
 'EAP',
 'MWS',
 'MWS',
 'MWS',
 'EAP',
 'EAP',
 'EAP',
 'MWS',
 'EAP',
 'EAP',
 'MWS',
 'HPL',


In [19]:
y_test.tolist()

['EAP',
 'MWS',
 'MWS',
 'EAP',
 'HPL',
 'MWS',
 'EAP',
 'EAP',
 'EAP',
 'EAP',
 'HPL',
 'HPL',
 'EAP',
 'HPL',
 'HPL',
 'EAP',
 'EAP',
 'HPL',
 'EAP',
 'MWS',
 'EAP',
 'EAP',
 'MWS',
 'EAP',
 'MWS',
 'EAP',
 'EAP',
 'MWS',
 'EAP',
 'HPL',
 'MWS',
 'HPL',
 'MWS',
 'MWS',
 'MWS',
 'EAP',
 'MWS',
 'MWS',
 'MWS',
 'EAP',
 'EAP',
 'MWS',
 'MWS',
 'MWS',
 'EAP',
 'EAP',
 'HPL',
 'HPL',
 'HPL',
 'MWS',
 'HPL',
 'MWS',
 'EAP',
 'HPL',
 'EAP',
 'MWS',
 'HPL',
 'HPL',
 'HPL',
 'MWS',
 'EAP',
 'HPL',
 'MWS',
 'MWS',
 'EAP',
 'EAP',
 'MWS',
 'MWS',
 'HPL',
 'EAP',
 'MWS',
 'EAP',
 'EAP',
 'EAP',
 'MWS',
 'MWS',
 'MWS',
 'HPL',
 'EAP',
 'EAP',
 'HPL',
 'MWS',
 'MWS',
 'HPL',
 'HPL',
 'HPL',
 'EAP',
 'EAP',
 'HPL',
 'EAP',
 'MWS',
 'MWS',
 'EAP',
 'MWS',
 'MWS',
 'EAP',
 'EAP',
 'MWS',
 'MWS',
 'EAP',
 'HPL',
 'MWS',
 'HPL',
 'MWS',
 'HPL',
 'MWS',
 'MWS',
 'MWS',
 'EAP',
 'HPL',
 'HPL',
 'MWS',
 'EAP',
 'EAP',
 'MWS',
 'MWS',
 'HPL',
 'EAP',
 'EAP',
 'EAP',
 'MWS',
 'EAP',
 'EAP',
 'HPL',
 'HPL',


In [30]:
#https://pandas.pydata.org/docs/reference/api/pandas.Series.eq.html
compared_test = y_test.eq(err_check) #this gives booleans
compared_test.tolist()

[True,
 True,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 Tr

In [32]:
#indexes of the falses: 2,5,6,11,12,13,17,30,41,46
#checking the indexes of the lists above
indexes = [2,5,6,11,12,13,17,30,41,46]

from operator import itemgetter
print(itemgetter(*indexes)(err_check)) #proba x-test values
print(itemgetter(*indexes)(y_test.tolist())) #actual values

('HPL', 'EAP', 'MWS', 'EAP', 'MWS', 'EAP', 'MWS', 'EAP', 'EAP', 'EAP')
('MWS', 'MWS', 'EAP', 'HPL', 'EAP', 'HPL', 'HPL', 'MWS', 'MWS', 'HPL')


In [13]:
false_s = y_test.eq(err_check)[~compared_test] #the ~ acts as a not symbol or !
ten_fs = false_s[:10] #first 10 data points that are "False"
ten_fs #can see the ids

id
id16303    False
id14743    False
id07281    False
id18564    False
id13058    False
id15700    False
id18277    False
id13971    False
id27104    False
id22092    False
Name: author, dtype: bool

In [164]:
ten_list = []
for x in X_test.index:
    for y in ten_fs.index:
        if x == y:
            ten_list.append(X_test.loc[x])
        else:
            continue
            
ten_df = pd.DataFrame(ten_list)
ten_df
#-----ending error analysis here --- incomplete - did not get feature weights

,processed,length,words,words_not_stopword,avg_word_length,commas,adjectives,nouns,verbs
id16303,he had seen so many customs and witnessed so g...,399,69,33,6.909091,1,7,10,1
id14743,she listened to me as she had done to the narr...,279,56,18,6.722222,5,2,7,1
id07281,his chief amusements were gunning and fishing ...,214,35,17,7.470588,2,1,7,1
id18564,johns i bade the knocker enter but was answere...,70,14,7,5.714286,2,0,6,0
id13058,at fifteen or even at twenty one for i had now...,133,27,13,5.461538,1,3,4,0
id15700,though not as yet licenced physicians we now h...,145,25,12,7.000000,2,1,2,0
id18277,the tide had turned and was coming in now and ...,89,19,7,5.428571,1,0,2,1
id13971,burkes reflections on the french revolution,43,6,4,8.250000,0,1,1,0
id27104,my best girl he had said relieves me from thes...,58,11,5,6.000000,2,0,1,0
id22092,that i had previously overestimated our depth ...,138,25,13,6.307692,1,3,4,2


In [126]:
#pipeline.get_params().keys()
#pipeline.get_params() #to check the values of the parameters

In [31]:
from sklearn.model_selection import GridSearchCV

hyperparameters = { 'features__text__tfidf__max_df': [0.9, 0.95],
                    'features__text__tfidf__min_df': [.005, 0.01], #https://stackoverflow.com/questions/27697766/understanding-min-df-and-max-df-in-scikit-countvectorizer
                    'features__text__tfidf__ngram_range': [(1,1), (1,2)],
                    'classifier__C': [0.01, 0.1, 1.0, 10],
                    'classifier__penalty': ['l2']
                  }
clf = GridSearchCV(pipeline, hyperparameters, cv=2)
 
# Fit and tune model
clf.fit(X_train, y_train)

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text',
                                                                        Pipeline(steps=[('selector',
                                                                                         TextSelector(key='processed')),
                                                                                        ('tfidf',
                                                                                         TfidfVectorizer(stop_words='english'))])),
                                                                       ('length',
                                                                        Pipeline(steps=[('selector',
                                                                                         NumberSelector(key='length')),
                                                                                        ('stan

In [32]:
clf.best_params_

{'classifier__C': 1.0,
 'classifier__penalty': 'l2',
 'features__text__tfidf__max_df': 0.9,
 'features__text__tfidf__min_df': 0.005,
 'features__text__tfidf__ngram_range': (1, 2)}

In [33]:
clf.refit

preds = clf.predict(X_test)

np.mean(preds == y_test)

0.6306097183534509

In [34]:
probs = clf.predict_proba(X_test)
print(probs)

[[0.66713647 0.12180309 0.21106044]
 [0.35136344 0.19196647 0.45667009]
 [0.08082458 0.75978111 0.15939431]
 ...
 [0.28970908 0.22561689 0.48467403]
 [0.85697726 0.08525126 0.05777148]
 [0.76387347 0.06255935 0.17356718]]


In [35]:
clf_ten = GridSearchCV(pipeline, hyperparameters, cv=10)
clf_ten.fit(X_train, y_train)
print(clf_ten.best_params_)
clf_ten.refit

preds_ten = clf_ten.predict(X_test)

np.mean(preds_ten == y_test)

{'classifier__C': 1.0, 'classifier__penalty': 'l2', 'features__text__tfidf__max_df': 0.9, 'features__text__tfidf__min_df': 0.005, 'features__text__tfidf__ngram_range': (1, 2)}


0.6306097183534509

In [36]:
probs_ten = clf_ten.predict_proba(X_test)
print(probs_ten)

[[0.66713647 0.12180309 0.21106044]
 [0.35136344 0.19196647 0.45667009]
 [0.08082458 0.75978111 0.15939431]
 ...
 [0.28970908 0.22561689 0.48467403]
 [0.85697726 0.08525126 0.05777148]
 [0.76387347 0.06255935 0.17356718]]


In [37]:
clf_twenty = GridSearchCV(pipeline, hyperparameters, cv=20)
clf_twenty.fit(X_train, y_train)
print(clf_twenty.best_params_)
clf_twenty.refit

preds_twenty = clf_twenty.predict(X_test)

np.mean(preds_twenty == y_test)

{'classifier__C': 10, 'classifier__penalty': 'l2', 'features__text__tfidf__max_df': 0.9, 'features__text__tfidf__min_df': 0.005, 'features__text__tfidf__ngram_range': (1, 2)}


0.6289074589910244

In [38]:
probs_twenty = clf_twenty.predict_proba(X_test)
print(probs_twenty)

[[0.70106009 0.09880416 0.20013575]
 [0.34960319 0.18623683 0.46415997]
 [0.06894585 0.78184954 0.14920462]
 ...
 [0.28201013 0.22023484 0.49775504]
 [0.87546349 0.077126   0.04741051]
 [0.78557142 0.05239929 0.16202929]]


In [39]:
clf_best_est = clf.best_estimator_.named_steps["classifier"]
clf_best_est.intercept_

array([ 0.53807808, -0.38115745, -0.15692062])

In [40]:
clf_ten_best_est = clf_ten.best_estimator_.named_steps["classifier"]
clf_ten_best_est.intercept_

array([ 0.53807808, -0.38115745, -0.15692062])

In [41]:
clf_twenty_best_est = clf_twenty.best_estimator_.named_steps["classifier"]
clf_twenty_best_est.intercept_

array([ 0.58587783, -0.39367748, -0.19220035])

In [42]:
coef = clf_best_est.coef_
print(coef)
print(coef.shape)

[[ 4.14719540e-01 -2.08516497e+00 -5.29878652e-01  2.43643893e-01
   1.78655758e+00 -1.12740368e+00  4.16938868e-01 -6.23595119e-02
  -2.24259895e-01  7.99976457e-01 -7.95025854e-01  3.73724990e-02
  -5.85526297e-02 -9.28703714e-01  3.16504763e-01 -4.97738441e-02
   1.24903739e-01 -1.38846101e-01  3.84644373e-02 -9.14970984e-02
   6.43978868e-01 -4.57907060e-01  1.51199087e-01 -5.55312536e-01
   2.99680833e-01  1.67003062e-02  9.54065532e-02  9.44776188e-01
  -8.23426134e-01  1.40879510e+00 -1.34117241e+00  8.22101310e-01
  -6.36252562e-01 -8.54586274e-02 -1.05939785e+00 -1.11121751e+00
   3.59705438e-01 -3.77114306e-02 -1.36971230e+00  3.00810468e-01
  -6.87412647e-01 -2.89221020e-01 -4.62588346e-01 -1.00232564e-01
  -6.13404968e-01 -7.83692316e-01 -1.55166853e-01 -1.29056552e-01
   6.00728876e-02 -6.45469538e-01 -5.67175946e-01 -1.03140342e-01
  -9.40292494e-01  5.43187889e-01  3.35858540e-02 -3.09247567e-01
   1.08127573e+00 -6.55150177e-01 -1.00045919e+00 -4.24589961e-01
   7.19461

In [43]:
coef_ten = clf_ten_best_est.coef_
print(coef_ten)
print(coef_ten.shape)

[[ 4.14719540e-01 -2.08516497e+00 -5.29878652e-01  2.43643893e-01
   1.78655758e+00 -1.12740368e+00  4.16938868e-01 -6.23595119e-02
  -2.24259895e-01  7.99976457e-01 -7.95025854e-01  3.73724990e-02
  -5.85526297e-02 -9.28703714e-01  3.16504763e-01 -4.97738441e-02
   1.24903739e-01 -1.38846101e-01  3.84644373e-02 -9.14970984e-02
   6.43978868e-01 -4.57907060e-01  1.51199087e-01 -5.55312536e-01
   2.99680833e-01  1.67003062e-02  9.54065532e-02  9.44776188e-01
  -8.23426134e-01  1.40879510e+00 -1.34117241e+00  8.22101310e-01
  -6.36252562e-01 -8.54586274e-02 -1.05939785e+00 -1.11121751e+00
   3.59705438e-01 -3.77114306e-02 -1.36971230e+00  3.00810468e-01
  -6.87412647e-01 -2.89221020e-01 -4.62588346e-01 -1.00232564e-01
  -6.13404968e-01 -7.83692316e-01 -1.55166853e-01 -1.29056552e-01
   6.00728876e-02 -6.45469538e-01 -5.67175946e-01 -1.03140342e-01
  -9.40292494e-01  5.43187889e-01  3.35858540e-02 -3.09247567e-01
   1.08127573e+00 -6.55150177e-01 -1.00045919e+00 -4.24589961e-01
   7.19461

In [44]:
coef_twenty = clf_twenty_best_est.coef_
print(coef_twenty)
print(coef_twenty.shape)

[[ 4.37959871e-01 -3.69862413e+00 -6.61166256e-01  2.69999468e-01
   2.66033547e+00 -1.31816282e+00  4.85723517e-01 -7.52992656e-02
  -2.64553725e-01  9.44259454e-01 -8.86477706e-01  8.40288180e-02
  -5.89289614e-02 -1.03121098e+00  3.54795347e-01 -1.02408185e-01
   1.12935834e-01 -1.61522609e-01  8.55246069e-02 -1.17165771e-01
   7.02472120e-01 -5.13031650e-01  1.54379508e-01 -5.93734642e-01
   3.08075120e-01  4.07004055e-02  3.37585616e-01  1.08992068e+00
  -9.37827728e-01  1.68596483e+00 -1.64511671e+00  9.68067806e-01
  -7.06127623e-01 -1.14673184e-01 -1.27158417e+00 -1.18995537e+00
   4.26052487e-01  1.49107647e-01 -1.59508485e+00  2.88099008e-01
  -7.86982087e-01 -3.42249405e-01 -5.07792040e-01 -1.14157298e-01
  -7.02425976e-01 -4.33726434e-01 -1.47844480e-01 -1.63560235e-01
   4.49273911e-02 -6.33103033e-01 -5.97817983e-01  4.41174281e-02
  -1.09369643e+00  6.58055675e-01  1.88709603e-03 -3.56855501e-01
   1.25438837e+00 -7.28514233e-01 -1.17337492e+00 -4.63951717e-01
   5.86667

In [45]:
new_df_coef = pd.DataFrame(zip(X_train.columns, np.transpose(coef)), columns=['features', 'coef'])
new_df_coef_ten = pd.DataFrame(zip(X_train.columns, np.transpose(coef_ten)), columns=['features', 'coef'])
new_df_coef_twenty = pd.DataFrame(zip(X_train.columns, np.transpose(coef_twenty)), columns=['features', 'coef'])
print(new_df_coef)
print(new_df_coef_ten)
print(new_df_coef_twenty)
#by transposing, it shows that the first 7 items are the first 7 features
#left to right in the column are the classes EAP, HPL, MWS

             features                                               coef
0           processed  [0.4147195397595956, -0.39444750082021424, -0....
1              length  [-2.0851649711135485, -1.4021922528930457, 3.4...
2               words  [-0.5298786516198994, 0.29321147039977813, 0.2...
3  words_not_stopword  [0.24364389253437305, -0.8395103150655907, 0.5...
4     avg_word_length  [1.7865575833969851, -0.09716796445490702, -1....
5              commas  [-1.127403679039466, 1.3987650325554917, -0.27...
6          adjectives  [0.41693886750844505, -0.6397927611913947, 0.2...
7               nouns  [-0.06235951187129047, -0.33902243992088943, 0...
8               verbs  [-0.22425989459081946, -0.16932780617307824, 0...
             features                                               coef
0           processed  [0.4147195397595956, -0.39444750082021424, -0....
1              length  [-2.0851649711135485, -1.4021922528930457, 3.4...
2               words  [-0.5298786516198994, 0.2932

In [1]:
#CV=2, CV=10
#3 most important features (absolute values):
#EAP: length, avg_word_length, commas
#HPL: length, commas, words_not_stopwords
#MWS: length, avg_word_length, words_not_stopwords

#CV=20
#3 most important features (absolute values):
#EAP: length, avg_word_length, commas
#HPL: length, commas, words_not_stopwords
#MWS: length, avg_word_length, words_not_stopwords

#CV=2, CV=10
#3 least important features (absolute values):
#EAP: nouns, verbs, words_not_stopwords
#HPL: avg_word_length, verbs, words
#MWS: processed, words, adjectives

#CV=20
#3 least important features (absolute values):
#EAP: nouns, verbs, words_not_stopwords
#HPL: avg_word_length, verbs, words
#MWS: processed, words, adjectives

In [49]:
print(new_df_coef.loc[0][1]) #checking what is in a row

[ 0.41471954 -0.3944475  -0.02027204]
